In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import re


In [4]:
data = pd.read_csv("/Users/sonn/Sonn/Workspace/Projects/IonBatteryQML/data/MaterialsProject/MaterialsProjectLiIonBattery.csv")

In [5]:
data.head(3)

,battery_id,id_charge,id_discharge,battery_formula,framework_formula,formula_discharge,formula_charge,working_ion,V_avg,capacity_grav,capacity_vol,energy_grav,energy_vol,max_delta_volume,stability_charge,stability_discharge
0,mp-28_Li,mp-28,mp-1185231,Li0-3Ce,Ce,Li3Ce,Ce,Li,-0.444846,499.595769,1295.168213,-222.242937,-576.149771,3.042358,0.000000,0.333634
1,mp-573471_Li,mp-573471,mp-1198729,Li4.25-4.4Sn,Sn,Li22Sn5,Li17Sn4,Li,-0.840299,26.936090,69.584747,-22.634379,-58.472018,0.009450,0.000000,0.010070
2,mp-80_Li,mp-80,mp-7955,Li0-3Sb,Sb,Li3Sb,Sb,Li,1.099595,563.913254,1701.062460,620.075950,1870.479038,1.863614,0.290332,0.000000


In [6]:

# Dữ liệu gốc từ ảnh
data = {
    'alloy_formula': data["formula_discharge"]
}

In [7]:

# Tạo DataFrame cơ bản
df = pd.DataFrame(data)

# Từ điển thuộc tính nguyên tố (các giá trị quan trọng cho ML)
element_properties = {
    'Al': {'atomic_number': 13, 'atomic_mass': 26.98, 'electronegativity': 1.61, 'melting_point': 933.47, 
           'density': 2.70, 'atomic_radius': 143, 'valence_electrons': 3, 'period': 3, 'group': 13},
    'Cu': {'atomic_number': 29, 'atomic_mass': 63.55, 'electronegativity': 1.90, 'melting_point': 1357.77,
           'density': 8.96, 'atomic_radius': 128, 'valence_electrons': 11, 'period': 4, 'group': 11},
    'Mn': {'atomic_number': 25, 'atomic_mass': 54.94, 'electronegativity': 1.55, 'melting_point': 1519,
           'density': 7.21, 'atomic_radius': 127, 'valence_electrons': 7, 'period': 4, 'group': 7},
    'Mo': {'atomic_number': 42, 'atomic_mass': 95.95, 'electronegativity': 2.16, 'melting_point': 2896,
           'density': 10.28, 'atomic_radius': 139, 'valence_electrons': 6, 'period': 5, 'group': 6},
    'Re': {'atomic_number': 75, 'atomic_mass': 186.21, 'electronegativity': 1.9, 'melting_point': 3459,
           'density': 21.02, 'atomic_radius': 137, 'valence_electrons': 7, 'period': 6, 'group': 7},
    'V': {'atomic_number': 23, 'atomic_mass': 50.94, 'electronegativity': 1.63, 'melting_point': 2183,
           'density': 6.0, 'atomic_radius': 134, 'valence_electrons': 5, 'period': 4, 'group': 5},
    'Sb': {'atomic_number': 51, 'atomic_mass': 121.76, 'electronegativity': 2.05, 'melting_point': 903.78,
           'density': 6.70, 'atomic_radius': 145, 'valence_electrons': 5, 'period': 5, 'group': 15},
    'Fe': {'atomic_number': 26, 'atomic_mass': 55.85, 'electronegativity': 1.83, 'melting_point': 1811,
           'density': 7.87, 'atomic_radius': 126, 'valence_electrons': 8, 'period': 4, 'group': 8},
    'W': {'atomic_number': 74, 'atomic_mass': 183.84, 'electronegativity': 2.36, 'melting_point': 3695,
           'density': 19.25, 'atomic_radius': 139, 'valence_electrons': 6, 'period': 6, 'group': 6},
    'Sn': {'atomic_number': 50, 'atomic_mass': 118.71, 'electronegativity': 1.96, 'melting_point': 505.08,
           'density': 7.29, 'atomic_radius': 141, 'valence_electrons': 4, 'period': 5, 'group': 14},
    'Ti': {'atomic_number': 22, 'atomic_mass': 47.87, 'electronegativity': 1.54, 'melting_point': 1941,
           'density': 4.51, 'atomic_radius': 147, 'valence_electrons': 4, 'period': 4, 'group': 4},
    'Ni': {'atomic_number': 28, 'atomic_mass': 58.69, 'electronegativity': 1.91, 'melting_point': 1728,
           'density': 8.91, 'atomic_radius': 124, 'valence_electrons': 10, 'period': 4, 'group': 10},
    'O': {'atomic_number': 8, 'atomic_mass': 15.999, 'electronegativity': 3.44, 'melting_point': 54.36,
          'density': 0.001429, 'atomic_radius': 66, 'valence_electrons': 6, 'period': 2, 'group': 16}
}

def parse_formula(formula):
    """Phân tích công thức hóa học để trích xuất nguyên tố và tỷ lệ"""
    # Pattern để tìm nguyên tố và số lượng
    pattern = r'([A-Z][a-z]?)(\d*)'
    matches = re.findall(pattern, formula)
    
    composition = {}
    total_atoms = 0
    
    for element, count in matches:
        count = int(count) if count else 1
        composition[element] = count
        total_atoms += count
    
    # Tính tỷ lệ phần trăm
    composition_percent = {elem: (count/total_atoms)*100 for elem, count in composition.items()}
    
    return composition, composition_percent, total_atoms

def extract_features(df):
    """Trích xuất đặc trưng chuyên sâu từ công thức hợp kim"""
    
    features = []
    
    for formula in df['alloy_formula']:
        composition, composition_percent, total_atoms = parse_formula(formula)
        
        # Đặc trưng cơ bản
        feature_dict = {
            'formula': formula,
            'num_elements': len(composition),
            'total_atoms': total_atoms,
        }
        
        # Tỷ lệ từng nguyên tố (one-hot encoding style)
        all_elements = set()
        for f in df['alloy_formula']:
            comp, _, _ = parse_formula(f)
            all_elements.update(comp.keys())
        
        for element in sorted(all_elements):
            feature_dict[f'{element}_percent'] = composition_percent.get(element, 0.0)
            feature_dict[f'{element}_count'] = composition.get(element, 0)
        
        # Đặc trưng dựa trên thuộc tính nguyên tố
        if composition:
            # Trọng số trung bình các thuộc tính
            prop_names = ['atomic_number', 'atomic_mass', 'electronegativity', 'melting_point', 
                         'density', 'atomic_radius', 'valence_electrons', 'period', 'group']
            
            for prop in prop_names:
                weighted_sum = 0
                total_weight = 0
                values = []
                
                for element, count in composition.items():
                    if element in element_properties:
                        prop_value = element_properties[element][prop]
                        weighted_sum += prop_value * count
                        total_weight += count
                        values.append(prop_value)
                
                if total_weight > 0:
                    feature_dict[f'avg_{prop}'] = weighted_sum / total_weight
                    feature_dict[f'max_{prop}'] = max(values) if values else 0
                    feature_dict[f'min_{prop}'] = min(values) if values else 0
                    feature_dict[f'std_{prop}'] = np.std(values) if len(values) > 1 else 0
                    feature_dict[f'range_{prop}'] = max(values) - min(values) if values else 0
                else:
                    feature_dict[f'avg_{prop}'] = 0
                    feature_dict[f'max_{prop}'] = 0
                    feature_dict[f'min_{prop}'] = 0
                    feature_dict[f'std_{prop}'] = 0
                    feature_dict[f'range_{prop}'] = 0
        
        # Đặc trưng phức tạp hơn
        if len(composition) >= 2:
            elements = list(composition.keys())
            
            # Chênh lệch độ âm điện (quan trọng cho liên kết)
            electroneg_values = []
            for elem in elements:
                if elem in element_properties:
                    electroneg_values.append(element_properties[elem]['electronegativity'])
            
            if len(electroneg_values) >= 2:
                feature_dict['electronegativity_diff_max'] = max(electroneg_values) - min(electroneg_values)
            else:
                feature_dict['electronegativity_diff_max'] = 0
            
            # Tỷ lệ kích thước nguyên tử
            radius_values = []
            for elem in elements:
                if elem in element_properties:
                    radius_values.append(element_properties[elem]['atomic_radius'])
            
            if len(radius_values) >= 2:
                feature_dict['size_ratio'] = max(radius_values) / min(radius_values) if min(radius_values) > 0 else 1
            else:
                feature_dict['size_ratio'] = 1
        else:
            feature_dict['electronegativity_diff_max'] = 0
            feature_dict['size_ratio'] = 1
        
        # Đặc trưng về cấu trúc
        feature_dict['has_transition_metal'] = any(
            elem in ['Ti', 'V', 'Mn', 'Fe', 'Ni', 'Cu', 'Mo', 'W', 'Re'] 
            for elem in composition.keys()
        )
        
        feature_dict['has_oxygen'] = 'O' in composition
        feature_dict['has_aluminum'] = 'Al' in composition
        
        # Tỷ lệ Al (quan trọng vì hầu hết là hợp kim Al)
        feature_dict['al_dominance'] = composition_percent.get('Al', 0.0)
        
        # Entropy cấu hình (đo độ phức tạp của hợp kim)
        entropy = 0
        for fraction in composition_percent.values():
            if fraction > 0:
                entropy -= (fraction/100) * np.log(fraction/100)
        feature_dict['configurational_entropy'] = entropy
        
        features.append(feature_dict)
    
    return pd.DataFrame(features)

# Tạo DataFrame với đặc trưng
feature_df = extract_features(df)

print("Shape của DataFrame:", feature_df.shape)
print("\nCác cột đặc trưng:")
print(feature_df.columns.tolist())

print("\nMẫu dữ liệu (5 dòng đầu):")
print(feature_df.head())

print("\nThống kê mô tả:")
print(feature_df.describe())

# Chuẩn bị dữ liệu cho deep learning
def prepare_for_deep_learning(df):
    """Chuẩn bị dữ liệu cho mô hình deep learning"""
    
    # Tách các cột số và phân loại
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    boolean_cols = [col for col in df.columns if df[col].dtype == bool]
    categorical_cols = [col for col in df.columns if col not in numeric_cols + boolean_cols + ['formula']]
    
    # Tạo bản sao để xử lý
    processed_df = df.copy()
    
    # Chuyển boolean thành int
    for col in boolean_cols:
        processed_df[col] = processed_df[col].astype(int)
    
    # Xử lý categorical nếu có
    le = LabelEncoder()
    for col in categorical_cols:
        if col != 'formula':
            processed_df[col + '_encoded'] = le.fit_transform(processed_df[col].astype(str))
    
    # Chuẩn hóa dữ liệu số
    feature_cols = [col for col in processed_df.columns if col not in ['formula'] + categorical_cols]
    
    scaler = StandardScaler()
    processed_df[feature_cols] = scaler.fit_transform(processed_df[feature_cols])
    
    return processed_df, feature_cols, scaler

# Chuẩn bị dữ liệu
processed_df, feature_columns, scaler = prepare_for_deep_learning(feature_df)

print(f"\n=== DÀNH CHO DEEP LEARNING ===")
print(f"Số đặc trưng cuối cùng: {len(feature_columns)}")
print(f"Các đặc trưng chính: {feature_columns[:10]}...")

# Tạo ma trận đặc trưng cho neural network
X = processed_df[feature_columns].values
print(f"\nKích thước ma trận đặc trưng: {X.shape}")

# Lưu dữ liệu
print("\n=== LƯU DỮ LIỆU ===")
feature_df.to_csv('alloy_features_raw.csv', index=False)
processed_df.to_csv('alloy_features_processed.csv', index=False)
np.save('alloy_features_matrix.npy', X)

print("✅ Đã lưu:")
print("- alloy_features_raw.csv: Đặc trưng gốc")  
print("- alloy_features_processed.csv: Đặc trưng đã xử lý")
print("- alloy_features_matrix.npy: Ma trận numpy cho DL")

# Hiển thị correlation matrix của một số đặc trưng quan trọng
import matplotlib.pyplot as plt
import seaborn as sns

key_features = ['avg_atomic_number', 'avg_melting_point', 'avg_density', 
                'electronegativity_diff_max', 'configurational_entropy', 
                'al_dominance', 'num_elements']

print(f"\nTương quan giữa các đặc trưng chính:")
correlation_matrix = feature_df[key_features].corr()
print(correlation_matrix)

Shape của DataFrame: (2434, 195)

Các cột đặc trưng:
['formula', 'num_elements', 'total_atoms', 'Ac_percent', 'Ac_count', 'Ag_percent', 'Ag_count', 'Al_percent', 'Al_count', 'As_percent', 'As_count', 'Au_percent', 'Au_count', 'B_percent', 'B_count', 'Ba_percent', 'Ba_count', 'Bi_percent', 'Bi_count', 'C_percent', 'C_count', 'Ca_percent', 'Ca_count', 'Cd_percent', 'Cd_count', 'Ce_percent', 'Ce_count', 'Cl_percent', 'Cl_count', 'Co_percent', 'Co_count', 'Cr_percent', 'Cr_count', 'Cs_percent', 'Cs_count', 'Cu_percent', 'Cu_count', 'Dy_percent', 'Dy_count', 'Er_percent', 'Er_count', 'Eu_percent', 'Eu_count', 'F_percent', 'F_count', 'Fe_percent', 'Fe_count', 'Ga_percent', 'Ga_count', 'Ge_percent', 'Ge_count', 'H_percent', 'H_count', 'Ho_percent', 'Ho_count', 'In_percent', 'In_count', 'Ir_percent', 'Ir_count', 'K_percent', 'K_count', 'La_percent', 'La_count', 'Li_percent', 'Li_count', 'Lu_percent', 'Lu_count', 'Mg_percent', 'Mg_count', 'Mn_percent', 'Mn_count', 'Mo_percent', 'Mo_count', 'N_p